In [1]:
import pandas as pd

In [2]:
files = ["defweb17_utf8.csv", "defweb18_utf8.csv", "defweb19_utf8.csv", "defweb20_0.csv", "defweb21_0.csv", "defweb22_0.csv", "defweb23.csv"]

In [3]:
causas_desnutricion = [f"E{i}" for i in range(40, 47)]
causas_desnutricion.extend([f"D{i}" for i in range(50, 54)])
causas_desnutricion.append("P05")

In [4]:
df_final = pd.read_csv(files[0])[['GRUPEDAD']].drop_duplicates().sort_values('GRUPEDAD')

for file in files:
    anio = file[6:][:2]
    
    df_temp = pd.read_csv(file, sep=None, engine='python')
       
    df_temp = df_temp[df_temp["CAUSA"].isin(causas_desnutricion)]
    
    resumen = df_temp.groupby("GRUPEDAD")["CUENTA"].sum().reset_index()
    resumen = resumen.rename(columns={"CUENTA": f"20{anio}"})
    
    df_final = pd.merge(df_final, resumen, on="GRUPEDAD", how="left")
    

df_final = df_final.fillna(0)
columnas_anios = ["2017","2018","2019","2020","2021","2022","2023"]
df_final[columnas_anios] = df_final[columnas_anios].astype(int)

print(df_final)

              GRUPEDAD  2017  2018  2019  2020  2021  2022  2023
0    01_Menor de 1 año    24    26    20    15    13    13    21
1             02_1 a 9    15     5     8     4    10     5     6
2           03_10 a 14     3     0     3     2     4     4     0
3           04_15 a 19     5     5     3     2     4     3     3
4           05_20 a 24     1     5     4     2     2     2     1
5           06_25 a 29     1     3     3     8     2     3     6
6           07_30 a 34     5     3     3     8     7     2     5
7           08_35 a 39     4     6     3     5     3     4     7
8           09_40 a 44     7    10     7    15    11     6    11
9           10_45 a 49    11     2     5     7     6     7     8
10          11_50 a 54    12     9     8     9    14    12    13
11          12_55 a 59    18    10    14    16    18    22    15
12          13_60 a 64    30    23    26    20    29    17    18
13          14_65 a 69    44    35    44    36    52    39    50
14          15_70 a 74   

In [5]:
totales = {}

for file in files:
    anio = file[6:][:2]
    
    df_temp = pd.read_csv(file, sep=None, engine='python')
    df_temp = df_temp[df_temp["CAUSA"].isin(causas_desnutricion)]
    totales[f"20{anio}"] = df_temp["CUENTA"].sum()

df_totales = pd.DataFrame.from_dict(totales, orient='index')
df_totales = df_totales.reset_index()
df_totales.columns = ['ANIO', 'TOTAL']
print(df_totales)


   ANIO  TOTAL
0  2017    885
1  2018    789
2  2019    855
3  2020    743
4  2021    859
5  2022    849
6  2023    880


In [6]:
totales_por_anio = df_final[columnas_anios].sum()
df_final.loc['TOTAL'] = totales_por_anio
df_final.loc['TOTAL', 'GRUPEDAD'] = 'Total'
df_final[columnas_anios] = df_final[columnas_anios].astype(int)
df_final = df_final.reset_index(drop=True)
print(df_final)

              GRUPEDAD  2017  2018  2019  2020  2021  2022  2023
0    01_Menor de 1 año    24    26    20    15    13    13    21
1             02_1 a 9    15     5     8     4    10     5     6
2           03_10 a 14     3     0     3     2     4     4     0
3           04_15 a 19     5     5     3     2     4     3     3
4           05_20 a 24     1     5     4     2     2     2     1
5           06_25 a 29     1     3     3     8     2     3     6
6           07_30 a 34     5     3     3     8     7     2     5
7           08_35 a 39     4     6     3     5     3     4     7
8           09_40 a 44     7    10     7    15    11     6    11
9           10_45 a 49    11     2     5     7     6     7     8
10          11_50 a 54    12     9     8     9    14    12    13
11          12_55 a 59    18    10    14    16    18    22    15
12          13_60 a 64    30    23    26    20    29    17    18
13          14_65 a 69    44    35    44    36    52    39    50
14          15_70 a 74   

In [ ]:
# Exportar a CSV
df_final.to_csv("desnutricion_2017-2023.csv", index=False, sep=";", encoding="utf-8-sig")

In [ ]:
# Exportar a Excel
df_final.to_excel("desnutricion_2017-2023.xlsx", index=False)